### Main notebook

In [1]:
import pandas as pd
import numpy as np
import json

import networkx as nx

from tqdm import tqdm

from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data

# read data from csv
cpds = pd.read_csv('../GNN_toxic/data/raw/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('data/pairs_final.csv', index_col=0)
cofactors = pd.read_csv('data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)

In [2]:
# Create a Compound object for each row in the DataFrame and add it to the data
for index, row in cpds.iterrows():
    entry = row['Entry']
    name = row['Names']
    formula = row['Formula']
    mw = row['mol_weight']
    smiles = row['SMILES']
    is_cofactor = row['Entry'] in cofactors['Entry'].values
    is_toxic = row['toxic']

    compound = Compound(entry, name, formula, mw, smiles, is_cofactor, is_toxic)
    data.add_element('compound', compound)

# Create a Reaction object for each row in the DataFrame and add it to the data
for index, row in rxns.iterrows():
    entry = row['Entry']
    name = row['Names']
    compounds = row['Compound']
    enzyme = row['EC Number']

    reaction = Reaction(entry, name, compounds, enzyme)
    data.add_element('reaction', reaction)

In [3]:
# number of times a metabolite apperas on pairs dataset
graph.get_number_of_occurences(pairs)
graph.create_graph(data=data, pairs=pairs)

# nodes: 8591 
# edges: 30026


100%|██████████| 8591/8591 [00:00<00:00, 515283.58it/s]


In [4]:
graph.calculate_edge_mol_weight(data)
graph.calculate_smiles_similarity(data)

 21%|██        | 6325/30026 [00:00<00:00, 62945.41it/s][17:17:34] WARNING: not removing hydrogen atom with dummy atom neighbors
[17:17:34] WARNING: not removing hydrogen atom with dummy atom neighbors
[17:17:34] WARNING: not removing hydrogen atom with dummy atom neighbors
[17:17:34] WARNING: not removing hydrogen atom with dummy atom neighbors
[17:17:34] WARNING: not removing hydrogen atom with dummy atom neighbors
[17:17:34] WARNING: not removing hydrogen atom with dummy atom neighbors
[17:17:34] WARNING: not removing hydrogen atom with dummy atom neighbors
[17:17:34] WARNING: not removing hydrogen atom with dummy atom neighbors
100%|██████████| 30026/30026 [00:26<00:00, 1140.24it/s]


In [5]:
######### VALIDATION SET FROM nicepath ###########
test_cases = pd.read_csv('data/original/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases['paths_list'] = test_cases['Pathway '].apply(lambda x: x.split(','))
paths = graph.validate(test_cases, 'smiles_similarity')

Correct pathway predictions: 16
Correct pathway predictions (%): 32.0


### Try cluster graph nodes

In [6]:
paths

,Pathway,Correct
0,"[C00082, C00811, C01197, C01494, C05619, C0048...",True
1,"[C00223, C00024, C19971, C19961, C19823, C0004...",False
2,"[C00811, C01197, C01494, C05619, C00482, C0117...",True
3,"[C00079, C00080, C00903]",False
4,"[C00223, C00323, C00406, C10443, C17749, C17750]",False
5,"[C06561, C00509, C16492, C01460]",True
6,"[C05903, C12249, C12634, C12635, C12636]",True
7,"[C01477, C01514, C03515, C12632, C04900]",True
8,"[C05903, C00389, C10107, C12633, C11620]",True
9,"[C05905, C08604, C12095, C12096, C16299]",False


### Try similarity based on SMILES

In [7]:
smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C00323').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))


smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C12096').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))

Similarity: 0.989145183175034
Similarity: 0.5102379634753735
